<div dir=rtl>

# עיבוד שפה טבעית (20225211) 2023-2024 - תרגיל 4

על ידי: פן אייל - 208722058, קורן אביטבול - 318796448

</div>

<div dir=rtl>

## התקנות

דבר ראשון, נתקין את החבילות הנדרשות. ל-`datasets` ו-`tokenizers` שהכרנו מתרגיל 2 מצטרפת `transformers`, גם היא מבית האגינגפייס, שבעזרתה נוכל לטעון את המודל המאומן ואת הטוקנייזר שלו ולהשתמש בהם על טקסט חופשי.
</div>

In [8]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install -U datasets

   ---------------------------------------- 0.0/510.5 kB ? eta -:--:--
   -- ------------------------------------ 30.7/510.5 kB 660.6 kB/s eta 0:00:01
   ------- -------------------------------- 92.2/510.5 kB 1.1 MB/s eta 0:00:01
   ----------------- ---------------------- 225.3/510.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------  501.8/510.5 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 510.5/510.5 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.9 MB ? eta -:--:--
    --------------------------------------- 0.6/24.9 MB 12.2 MB/s eta 0:00:02
   -- ------------------------------------- 1.6/24.9 MB 17.3 MB/s eta 0:00:02
   ----- ---------------------------------- 3.3/24.9 MB 23.8 MB/s eta 0:00:01
   -------- ------------------------------- 5.2/24.9 MB 27.5 MB/s eta 0:00:01
   ---------- ----------------------------- 6.6/24.9 MB 28.0 MB/s eta 0:00:01
   ---------- ----------------------------- 6.6/24.9 MB 28.0 MB/s eta 

<div dir=rtl>

נייבא חבילות ומודולים.
הרובוטריקים של האגינגפייס יודעים לדבר עם `torch` וכיוון שנוח לנו נעבוד איתו לצרכי עיבוד נוסף של הפלטים.

</div>

In [9]:
import numpy as np
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn.functional import softmax, log_softmax
from torch import exp

<div dir=rtl>

נטען באמצעות `datasets` את אסופת סיפורי הילדים שעבדנו איתה כדי לאמן מודל שיכוני מילים בשבוע 7. היום לא ממש נאמן כלום, אבל זה טוב שיש משפטים מן המוכן במקום להמציא.

</div>

In [10]:
stories = datasets.load_dataset("deven367/babylm-100M-children-stories")

Found cached dataset parquet (file://C:/Users/paney/.cache/huggingface/datasets/deven367___parquet/deven367--babylm-100M-children-stories-8f7a018d8919c1f0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

<div dir=rtl>

כאמור, אנחנו נעבוד עם מודל [GPT-2](https://huggingface.co/openai-community/gpt2) (הקישור מוביל לדף הסבר באתר האגינגפייס).
צריך לטעון בנפרד את המודל ואת ה**טוקנייזר שלו**, שהוא מודל-עזר שתפקידו להפוך טקסט גולמי לקלט מספרי עבור הרובוטריק. הטוקנייזר של GPT-2 אומן בשיטת קידוד זוגות בתים, או BPE, עליו תוכלו לקרוא בפרק 2.5.2 של סטנפורד, או בחומרי העזר של הקורס, או ממש לעומק בהסרטון [הזה](https://www.youtube.com/watch?v=zduSFxRajkE) שיצא אך החודש.

שימו לב שבניגוד לתרגיל הקודם, הפעם לא נאמן כלום ולא נגריל כלום ולכן לא צריך לקבע זרע אקראיות.

</div>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

<div dir=rtl>

פונקציית הייצוג של מודל בחבילת `transformers` נותנת פרטים על הארכיטקטורה של המודל בצורה מקוננת ונוחה.

**1. ענו על השאלות הבאות בהסתמך על הפלט של שני התאים הבאים (2 נק'):**
1. (0.25) מהו גודל אוצר המילים של המודל?
1. (0.75) `ModuleList` מתאר רשימת מודולים לפי סדר הפעלתם. מה שונה באופן סידור המודולים של בלוק השכבה ב-GPT-2 (כלומר, החלק שמשתכפל בהתאם למספר השכבות) מזה שלמדנו בכיתה? אין צורך להתבונן בתוך מרכיבי המודולים עצמם (כלומר ניתן להניח שמודול עם Attention בשם הוא בלוק צומי עצמי, למשל).
1. (1) שכבת ה-MLP של המודל משתמשת בפונקציית אקטיבציה שלא למדנו עליה. חקרו אודותיה מעט וענו מהי התכונה העיקרית שמבדילה אותה משלוש הפונקציות שאנחנו מכירים.

<hr>

**תשובות**:
1. אפשר לראות לפי שכבת הembedding כי גודל אוצר המילים הוא **50,257**
1. נוספה שכבת נרמול לפני שכבת הצומי, ובנוסף שכבת הfeedforward המלאה שונתה להיות multi layer perceptron. כלומר הרחבנו את שכבת הfeedforward להיות כמה שכבות.
1. התכונה העיקרית שמבדילה את הNewGELU מהפונקציות שלמדנו (ReLU, sigmoid, tanh) היא האי-מונוטוניות של הפונקציה.


</div>

In [ ]:
model

In [ ]:
tokenizer

<div dir=rtl>

נרענן את זכרוננו לגבי איך מחלצים את הטקסט מתוך הדאטא.

</div>

In [ ]:
story1 = list(stories['train'])[1]['text']
print(story1)

<div dir=rtl>

ועכשיו נבדוק איך הטוקנייזר מייצג אותו. הפונקציה הראשונה שנקרא לה תיצור עבורנו אצווה (batch) בפורמט פייטורץ (`pt`) שתכיל, בין היתר, את האלמנטים הבאים:
* קריאה לפונקצית `tokens()` תחזיר את המחרוזות אליהן התפצל הטקסט כדי להיכנס לאוצר המילים של המודל. BPE הוא טוקנייזר "תת-מילי" subword, כלומר יש מרכיבים שאינם מהווים מילים שלמות וזאת כדי שגם למילים נדירות או לא-מוכרות יהיה ייצוג. שימו לב שיש תו מיוחד שמוצג כאן כ-G עם נקודה מעל, תפקידו לסמן **רווח** והוא לרוב יתחבר למילה שאחריו.
* `input_ids` הם הטוקנים שיצר הטוקנייזר בצורתם הגולמית כאינדקסים מתוך אוצר המילים (כך, למשל, ניגש לשיכונים שלהם). אינטואיטיבית, נסו להעריך אם יש משמעות כלשהי לאינדקסים (אין סיבה שזה יהיה דווקא כך).

**2. ענו על השאלות הבאות (1):**
1. (0.5) מה מסמנים האלמנטים שמחזירה הפונקציה `word_ids()`, ובאיזה סיטואציה במיוחד הם חשובים? חישבו על הבעיות שלמדנו עד היום בקורס.
1. (0.5) בחרו עוד אלמנט בפלט של הטוקנייזר והסבירו את תפקידו. אפשר להדגים במקום המיועד לכך בתא הבא.

<hr>

**תשובות:**

1. האלמנטים שמחזירה הפונקציה word_ids() מסמנים את אינדקס המילה במשפט אליה הtoken שייך. דבר זה יהיה חשוב בסיטואציה בה נרצה לזהות מילים או כמה מילים בטקסט, כגון במשימת זיהוי ישוייות שבה המילים עצמן מעניינות אותנו, ולא הtokens שמייצגים את היישות.

1. הפלט של הטוקנייזר מכיל את האלמנט 'attention_mask' שהוא מערך בינרי בגודל שווה למספר הטוקנים בטקסט. עבור כל טוקן בטקסט, ערך של 1 ב-attention mask מציין שהטוקן אמור להשפיע על הפלט של המודל, בעוד שערך 0 מציין שהטוקן אינו משפיע.


</div>

In [ ]:
input_toks = tokenizer.batch_encode_plus([story1], return_tensors='pt')
tok_texts = input_toks.tokens()
print(' '.join(tok_texts[:5]))
print(len(input_toks.input_ids[0]))
print(input_toks.input_ids[0][:5])
print(input_toks.word_ids()[:5])

### 2.2 One more element: ###
print(input_toks['attention_mask'])

<div dir=rtl>

כעת נפעיל את המודל עצמו באמצעות קריאה ישירה אליו על המבנה המפורק (שתי כוכביות) של פלט הטוקנייזר.
הקריאה הזו תעביר את הטוקנים דרך כל הרובוטריק, מהשיכונים ועד שכבת החיזוי, ותיתן לנו את תוצרי כל אחד משלבי החיזוי של GPT-2 **כמודל שפה**.

הפלט מגדיר מבנה די מסובך (אפשר לראות תיעוד מלא באתר הדוקומנטציה או באמצעות קריאה למודל עם סימן שאלה אחריו בתא קוד במחברת).
ניכנס ישר למה שמעניין אותנו - הלוג'יטים (logits), שהם הציונים הגולמיים של כל הטוקנים באוצר המילים בכל אחד משלבי החיזוי, כלומר על כל תחילית של הקלט.
כך האיבר הרביעי במבנה הלוג'יטים ייתן את כל התחזיות עבור כל הטוקנים האפשריים בהינתן ארבעת הטוקנים הראשונים של הטקסט, כלומר את מרחב התחזיות לטוקן החמישי. לאחר שנעביר את הווקטור הזה דרך סופטמקס, נקבל את התחזיות האלה כ**התפלגות**.

</div>

In [ ]:
generative_outputs = model(**input_toks)
gen_out_logits = generative_outputs[0][0]
print(len(gen_out_logits))
print(len(gen_out_logits[3]))
print(float(gen_out_logits[3][1748]))

In [ ]:
softmaxed_outputs = softmax(generative_outputs[0][0][3], dim=0)  # specify a dimension or an annoying warning comes up
float(softmaxed_outputs[1748])

<div dir=rtl>

נראה שהמודל מעניק למילה האמיתית, city, הסתברות של כחצי אחוז להיות המילה הבאה.
זה לא מעט בהינתן גודל אוצר המילים הכללי, אבל האם יש מילים שהוא מעדיף?

**3. ענו (1):**
1. (0.5) כמה מילים מעדיף המודל על-פני city?
1. (0.5) מהי המילה לה הוא מעניק את ההסתברות הגבוהה ביותר? (**לא האינדקס: המילה עצמה**)

</div>

In [ ]:
### Answers for 3 (edit where necessary) ###

### Answer for 3.1 ###

bigger_then_city = np.sum([1 for i in range(len(softmaxed_outputs)) if softmaxed_outputs[i] > softmaxed_outputs[1748]])
print("1. The model preffer", bigger_then_city, "words more than the word 'city'")

### Answer for 3.2 ###

def best_token_id(sm_output) -> int:
  return np.argmax(sm_output.detach().numpy())


def token_from_index(idx: int) -> str:
  '''
  Hint: look at the tokenizer
  '''
  vocab = tokenizer.get_vocab()
  return list(vocab.keys())[list(vocab.values()).index(idx)]


best_in_outputs = best_token_id(softmaxed_outputs)
print("2.",token_from_index(best_in_outputs))

<div dir=rtl>

וכך נוכל לראות את ההסתברויות של כל המילים ברצף. שימו לב שיש הסתברויות מאוד גבוהות להמשכי המילים הנכונים כאשר הטוקנייזר חותך אותן, למשל בתוך המילה sapphires:

</div>

In [ ]:
word_probs = []
for loc, tok_idx in enumerate(input_toks.input_ids[0]):
  if loc < 1: continue  # think why!
  sm_loc = softmax(gen_out_logits[loc - 1], dim = 0)
  prob = sm_loc[tok_idx]
  word_probs.append((tok_texts[loc], prob))
print('; '.join([f'{txt}: {prob:.4f}' for txt, prob in word_probs]))

<div dir=rtl>

כמו שהזכרנו בכיתה, קשה למצוא מטריקות טובות להערכת חילול טקסט. מה שכן אפשר לעשות זה להעריך מודלים באמצעות מידת הסבירות הכוללת שהם נותנים לטקסט נתון. המטריקה, שגם מיתרגמת לפונקציית הפסד נוחה, היא **בלבלות**, או בלעז **[perplexity](https://en.wikipedia.org/wiki/Perplexity)**.
היא הגיעה אלינו מתורת האינפורמציה, והרעיון הכללי שבה הוא לחשב כמה המודל "מופתע" מהרצף שהוא נתקל בו, לעומת מה שהוא "רגיל" אליו.

להלן נוסחאתה:

$$ PPL(\mathbf{x}) = \exp\left\{-\frac{1}{N}\sum_i^N\log P(x_i|x_{<i})\right\} $$

בכל נקודה ברצף, ככל שלוג ההסתברות גבוה יותר כך המודל "ציפה" יותר למילה האמיתית. לאחר השלילה וההעלאה מחדש בחזקה, מקבלים מדד שככל שהערך שלו **גבוה** יותר, המודל **פחות טוב** בזיהוי הרצף. באופן מדויק יותר, ככל שהבלבלות גבוהות כך הרצף פחות סביר לפי המודל.

**4. ממשו את פונקציית הבלבלות וחשבו אותה עבור הרצף שלנו (2).**

שימו לב שלוג ההסתברות אינו הלוג'יט! (מהו כן הלוג'יט? חכו לתרגיל 5.) תצטרכו לעשות כאן שימוש בפונקציה שייבאנו מבעוד מועד ביחד עם הסופטמקס (ר' לעיל) שמחשבת לוג הסתברות באופן ישיר ומהיר וללא בעיות נומריות.

כמו כן, שימו לב לשינוי הקטנטן שנאלצנו לעשות בתא הקודם כתוצאה מכך ש-GPT לא מתחיל רצפים עם טוקן מיוחד, והתאימו את הנוסחא בשאלה הזו.

</div>

In [ ]:
def perplexity(logits, true_idxs) -> float:
  assert len(logits) == len(true_idxs)
  N = len(true_idxs)

  ### 4. keep going ###

  log_probs = log_softmax(logits, dim=-1)
  true_log_probs = log_probs[range(N), true_idxs]
  return exp(-true_log_probs.mean()).item()

print(perplexity(gen_out_logits, input_toks.input_ids[0]))

<div dir=rtl>

נסבר את האוזן עם חישוב אותה המטריקה עבור המקטע הבא של הסיפור. אפשר לראות אם המודל "ציפה" לו יותר או פחות מלמקטע הראשון.

ואפשר גם סתם על טקסט חופשי, כמו בתא שאחריו.

ולראות אם GPT-2 מכיר ביטויים מפורסמים באנגלית.

</div>

In [ ]:
story2 = list(stories['train'])[2]['text']
input_toks_2 = tokenizer.batch_encode_plus([story2], return_tensors='pt')
print(len(input_toks_2.tokens()))
print(' '.join(input_toks_2.tokens()))
generative_outputs_2 = model(**input_toks_2)[0][0]
print(perplexity(generative_outputs_2, input_toks_2.input_ids[0]))

In [ ]:
idiom = 'The rain in Spain stays mainly on the plain.'
idiom_toks = tokenizer.batch_encode_plus([idiom], return_tensors='pt')
print(' '.join(idiom_toks.tokens()))
gen_outs_idiom = model(**idiom_toks)[0][0]
perplexity(gen_outs_idiom, idiom_toks.input_ids[0])

In [ ]:
true_idiom_last_tok = idiom_toks.input_ids[0][8]
print(int(true_idiom_last_tok))
idiom_sm = softmax(gen_outs_idiom[7], dim=0)
print(float(idiom_sm[true_idiom_last_tok]))
print(float(idiom_sm.max()))

most_probable_token = int(best_token_id(idiom_sm))
print(most_probable_token, token_from_index(most_probable_token))

<div dir=rtl>

הפעם נבקש מ-GPT-2 להשלים רצפים בלי שאנחנו נותנים אותם כחלק מהקלט.
לא יהיה כאן שום הבדל מבחינת ההתנהגות, הודות לסגירת המשולש העליון של מטריצת הצומי, אבל ככה נדמה יותר טוב את אופן הפעולה של המודל **בשלב היישום** (שפוגשים היום עם הסייענים למיניהם): נותנים טקסט והמודל משלים.

**5. עבור שלושת הרצפים הבאים, מצאו את המילה ש-GPT-2 ישלים תחת מנגנון חילול חמדן (כלומר, את הטוקן הכי סביר) (1):**

</div>

In [ ]:
sentences_to_complete = ['Why is the sky',
                         "You ain't seen nothing",
                         'Question: When was Barack Obama elected President of the USA? Answer:']

In [ ]:
def most_prob_continuation(sentence: str) -> int:

  ### Q5 - edit here ###
  sentence_toks = tokenizer.batch_encode_plus([sentence], return_tensors='pt')
  gen_outs = model(**sentence_toks)[0][0]
  sm = softmax(gen_outs[-1], dim=0)
  return best_token_id(sm)


for s in sentences_to_complete:
  most_prob_cont = most_prob_continuation(s)
  print(int(most_prob_cont), token_from_index(most_prob_cont))

<div dir=rtl>

**6. התשובה לשאלה האחרונה היא לא מה שציפינו עבור תשובה בעלת טוקן אחד. מצאו החל מאיזה מקום בדירוג הטוקנים מקבלים תשובה מהסוג הרצוי (שנה) והאם זו השנה הנכונה. (1)**

רמז: יש פונקציה מובנית לטנזורים של פייטורץ' שתקל על פתרון השאלה הזו.

**בונוס: עוד דרך לקבל תשובה נכונה היא להזין את החודש הנכון כהמשך הקלט ולראות מה הפלט הבא. מיצאו את ההסתברות שבהנתן הקלט המקורי (לעיל) המודל יוציא *בדגימה* את החודש הנכון ולאחריו את השנה הנכונה. הראו את החישוב. (0.5)**

<div>

In [ ]:
### Q6 + Bonus (more cells possible) ###
def most_prob_continuation(sentence: str, target_type: str = 'year') -> int:
    sentence_toks = tokenizer.batch_encode_plus([sentence], return_tensors='pt')
    gen_outs = model(**sentence_toks)[0][0]
    sm = softmax(gen_outs[-1], dim=0)

    # Filter tokens based on the desired type
    target_type_ids = [i for i, token in enumerate(tokenizer.convert_ids_to_tokens(range(len(sm)))) if target_type in token.lower()]

    # Find the index of the highest probability token of the desired type
    best_type_token_id = max(target_type_ids, key=lambda x: sm[x].item())

    return best_type_token_id

# Function to get the token from its index
def token_from_index(index):
    return tokenizer.convert_ids_to_tokens(index)

# Find the correct year
correct_year_index = most_prob_continuation(sentences_to_complete[-1], target_type='year')
correct_year_token = token_from_index(correct_year_index)
print(correct_year_index, correct_year_token)
